### В этом файле содержится реализация метода Зайделя и его версия с последовательной релаксацией

In [1]:
from math import sqrt
import numpy as np
import pandas as pd

In [2]:
def seidel(A, b, eps=1e-9, omega=None, max_iterations=30000):
    n = len(A)
    x = np.zeros(n)

    converge = False
    step_counter = 0
    while not converge:
        x_new = np.copy(x)
        for i in range(n):
            s1 = sum(A[i][j] * x_new[j] for j in range(i))
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i][i]

        converge = np.linalg.norm(x - x_new) <= eps
        
        # если задана, то применяем релаксацию
        if omega:
            x = omega * x_new + (1 - omega) * x
        else:
            x = x_new
        step_counter += 1
        if step_counter >= max_iterations:
            print('Решение не сошлось по достижении максимального числа итераций.')
            break
    sol_b = np.dot(A, x)
    print('Понадобилось итераций для достижения условия сходимости: {}'.format(step_counter))
    print('Среднеквадратичная ошибка решения: {:.9f}'.format(np.linalg.norm(sol_b - b)))
    print("Является ли решение точным: {}".format(np.allclose(sol_b, b)))
    return x

Загрузим из файлов матрицы, на которых мы будем проверять корректность реализации.

In [3]:
delimiter = ','
well = np.loadtxt('well.txt', delimiter=delimiter)
random = np.loadtxt('random.txt', delimiter=delimiter)
poor = np.loadtxt('poor.txt', delimiter=delimiter)
b = np.loadtxt('right.txt', delimiter=delimiter)

In [4]:
print('Решаем обычным методом Зайделя\n')
print('Хорошо обусловленная матрица:')
well_x = seidel(well, b)
print('----------------------------')
print('Рандомная матрица:')
random_x = seidel(random, b)
print('----------------------------')
print('Очень плохо обусловленная матрица:')
poor_x = seidel(poor, b)
print('----------------------------')

Решаем обычным методом Зайделя

Хорошо обусловленная матрица:
Понадобилось итераций для достижения условия сходимости: 13
Среднеквадратичная ошибка решения: 0.000000001
Является ли решение точным: True
----------------------------
Рандомная матрица:
Понадобилось итераций для достижения условия сходимости: 1474
Среднеквадратичная ошибка решения: 0.000000000
Является ли решение точным: True
----------------------------
Очень плохо обусловленная матрица:
Решение не сошлось по достижении максимального числа итераций.
Понадобилось итераций для достижения условия сходимости: 30000
Среднеквадратичная ошибка решения: 0.269276147
Является ли решение точным: False
----------------------------


%%latex
**Как видим, на плохо обусловленной матрице возникают проблемы - сходимость не достигается. Попробуем улучшить результат с помощью последовательной релаксации, подобрав оптимальное значение $\omega$.**

In [ ]:
omegas = np.linspace(0, 2, 50)
poor_x_relaxed = None
poor_x_relaxed_min_err = None
best_omega = None
for omega in omegas:
    print('Коэффициент релаксации: {:.9f}'.format(omega))
    poor_x_relaxed_current = seidel(poor, b, omega=omega)
    err =  np.linalg.norm(np.dot(poor, poor_x_relaxed_current) - b)
    if not poor_x_relaxed_min_err or err < poor_x_relaxed_min_err:
        poor_x_relaxed = poor_x_relaxed_current
        poor_x_relaxed_min_err = err
        best_omega = omega
        print('Найдено новое лучшее значение коэффициента релаксации.')
    print('----------------------------')

Коэффициент релаксации: 0.000000000
Решение не сошлось по достижении максимального числа итераций.
Понадобилось итераций для достижения условия сходимости: 30000
Среднеквадратичная ошибка решения: 0.269276147
Является ли решение точным: False
Найдено новое лучшее значение коэффициента релаксации.
----------------------------
Коэффициент релаксации: 0.040816327
Решение не сошлось по достижении максимального числа итераций.
Понадобилось итераций для достижения условия сходимости: 30000
Среднеквадратичная ошибка решения: 0.468659459
Является ли решение точным: False
----------------------------
Коэффициент релаксации: 0.081632653
Решение не сошлось по достижении максимального числа итераций.
Понадобилось итераций для достижения условия сходимости: 30000
Среднеквадратичная ошибка решения: 0.405560971
Является ли решение точным: False
----------------------------
Коэффициент релаксации: 0.122448980
Решение не сошлось по достижении максимального числа итераций.
Понадобилось итераций для дост

После того, как нашли оптимальное значение коэффициента на плохо обусловленной матрице, убедимся, что на остальных матрицах решение все так же корректно работает.

In [ ]:
print('Решаем методом Зайделя с последовательной релаксацией c коэффициентом релаксации = {:.9f}\n'.format(best_omega))
print('Хорошо обусловленная матрица:')
well_x_relaxed = seidel(well, b, omega=best_omega)
print('----------------------------')
print('Рандомная матрица:')
random_x_relaxed = seidel(random, b, omega=best_omega)
print('----------------------------')
print('Очень плохо обусловленная матрица:')
poor_x_relaxed = seidel(poor, b, omega=best_omega)
print('----------------------------')

**Среднеквадратичная ошибка на плохо обусловленной матрице уменьшилась, хотя сходимости так и нет (или же она крайне медленная). На остальных матрицах ошибка почти не изменилась.**

Теперь сохраним наши результаты в таблицу.

In [ ]:
seidel_df = pd.DataFrame({'well': well_x, 'random': random_x, 'poor': poor_x})
seidel_relaxed_df = pd.DataFrame({'well': well_x_relaxed, 'random': random_x_relaxed, 'poor': poor_x_relaxed})

In [ ]:
seidel_df.to_csv('seidel.csv', index=None)
seidel_relaxed_df.to_csv('seidel_relaxed.csv', index=None)